In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [ ]:
model_name = "bert-base-uncased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
dataset = pd.read_csv('Tesla.csv')

In [ ]:
print(dataset.columns)

In [ ]:
def analyze_sentiment(news_text):
    inputs = tokenizer.encode_plus(news_text, return_tensors="pt", add_special_tokens=True, truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    sentiment_scores = torch.nn.functional.softmax(logits, dim=1).squeeze().tolist()
    return predicted_class, sentiment_scores

In [ ]:
def assess_risk(sentiment_class):
    sentiment_labels = ['Negative', 'Neutral', 'Positive']
    sentiment = sentiment_labels[sentiment_class]

    if sentiment == 'Positive':
        return "Low Risk"
    elif sentiment == 'Negative':
        return "High Risk"
    else:
        return "Medium Risk"

In [ ]:
for index, row in dataset.iterrows():
    news_text = row['News']
    sentiment_class, sentiment_scores = analyze_sentiment(news_text)
    risk_assessment = assess_risk(sentiment_class)
    
    print(f"News Text: {news_text}")
    #print(f"Sentiment Scores: {sentiment_scores}")
    print(f"Predicted Sentiment: {sentiment_class}")
    print(f"Risk Assessment: {risk_assessment}")
    print("=" * 50)